# Lede Algorithms, Lecture 4. 
#### July 19, 2019. Friday. 

## Introduction to Logistic Regression

We're going to be using logistic regression on a very very old albeit familiar dataset just to try and understand the various concepts around how we approach logistic regression problems. Logistic regression, which you may remember, is a classification algorithm. For the purpose of this class, we're restricting the scope to dichotomous or binary dependent variables, i.e. the categories your algorithm will predict will be one of two choices. The dataset: Titanic. The output: Survived? Yes or no. 

You might wonder why we are bothering to use this dataset again. Is it particularly interesting? And, what are the odds—no puns intended—of us ever using a dataset like this for journalistic reasons? Let's go through the exercise and then discuss the ethics of doing something like this on more current/live data. 

In [ ]:
# Let's import all the packages/modules that we need. 
# If you get a `ModuleNotFoundError`, run a `pip install` for the module 
# that failed to import.
# You'll need to have the following Python modules installed:
# - matplotlib
# - numpy
# - pandas
# - seaborn
# - sklearn

import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics import confusion_matrix

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style

### Loading the data in. 
You should notice that we now have two different files: `train.csv` and `test.csv`. Wait, what, why? (This data has been downloaded from Kaggle: https://www.kaggle.com/c/titanic/data)

`train.csv` is our training dataset, i.e. the dataset our algorithm will learn from. It has a bunch of features and the output label (i.e. survived: yes/no) for a subset of the passengers. The remaining passengers are in the `test.csv` file, i.e. they make up our test dataset. 

In [ ]:
# Yes, we are doing this here, but remember, relative paths are evil. 



# In theory, both dataframes should have the same number of columns, but 
# the training dataset should have more rows. Let's do a quick sanity check
# to see the kind of data we are dealing with. 


## Explore and clean the data
It's important to get a feel for the data, because unless you're not intimate with it, the chances of making a careless mistake are high. In this case, most of the column names are self-explanatory, but there are a couple that aren't. 
- `SibSp` defines family relationships for siblings and spouses
- `Parch` defines family relationships for parents and children; when 0, it means the child was travelling with a nanny. 

Once we understand what each of the columns mean, let's dive into the data. Typically, folks are 
inclined to do a `df.head()`, but the problem with that is it often masks inconsistencies, so randomly sampling data can be a better approach. It's not 100% foolproof, but...

In [ ]:
## Let's see a quick sample of our data to see what we've got. 



Now that we've had a quick glance, let's find out: 
- if any data is missing
- if any data needs to be normalised 
- if any data needs to be removed altogether
- ...and, let's make some decisions: do we really need all these columns?

In [ ]:
# First, we find attempt identifying missing data points. 




#### OK, so a few things: 
- Only three columns (henceforth called "features") are missing data. What are they, and what % of data is missing?


Note: anytime you think, "we can fudge this," **PAUSE**. 
What are the consequences of fudging the data? What would happen if you got it wrong? How much would it undermine everything else you've done? 


But, also, a passenger's PassengerID, name, or family relationships should hold no bearing on whether they survived the disaster or not. So, maybe we can drop those columns. 

In [ ]:
# let's drop the columns we don't need so that we can focus on what we do need. 
# remember: whatever action we take on the training data, we need to do exactly the same for the test data. 





### How do we solve the two missing "Embarked" values?

In [ ]:
# Let's first look at the data, to see if we can see anything obvious. 
# For example, can the family data help us extrapolate?
print(f"The NA indices for Embarked are at: {train_df.index[train_df['Embarked'].isna()]}")
train_df.iloc[[61, 829]]
      
      
      

In [ ]:
# Now, let's look at the missing data points, and see what we can do. 
# Starting with Embarked...
### What's the breakdown of values in Embarked?




# When we breakdown "Embarked" by the counts, we see 'S' is the most common, so we blindly adopt that. 
# We can blindly do this for two reasons:
# (i) the number of missing data points is tiiiiiiny 
# (ii) does the port where someone embarked _realistically_ impact their survival? 







### How do we solve the missing data in Cabin?

If you look through one of the references, you'll see that they use what they have and ignore what they don't. But, that doesn't seem entirely right, because it might skew our results one way or another. So, let's drop it. 


In [1]:
## Drop Cabin from our training and test data sets




### How do we solve the missing data in Age?

Age is obviously a far more crucial datapoint. The approaches adopted by the references vary. 
- Using the median age based on the data where age != NA
- Computing a random age based on the mean age and standard deviation

For the sake of simplicity, let's go down the route of option 1. Again, remember, we need to mimic this across the training & test datasets. 

In [ ]:
# When we fill the median age on the test dataframe, we stick to the median
# age we computed from the training data. This is because we shouldn't be
# doing _anything_ based on the test dataset. We might end up skewing the 
# results or overfitting, and we need to be careful to avoid that. 






## Data Normalisation
For logistic regression to work, the data needs to be "normalised". By that, we mean:
- we can't have string values, i.e. gender which is currently male/female should be numeric. 
- individual ages might lead to _overfitting_, so we create age brackets. 
- ...and we might want to see what we can do with the tickets, because they might be unique ticket IDs, which means it wouldn't help.

In [ ]:
# let's start with tickets, and see how many unique values we have






In [ ]:
# Right, you can immediately tell this isn't going to fly: 691 unique items here 
# will lead to overfitting. So, let's drop this. 






In [ ]:
# Next up, genders. Let's check what our gender counts are, and then decide 
# on unique values for each gender. 






In [ ]:
# OK, so that's pretty straightforward: add a new column to the dataframe: 'Male'. 
# If male, this column should have a value of 1. Else 0. 






In [ ]:
# And, now, let's do age brackets. How should we do this? 'Adult'/'Child' to
# keep it simple? Or, proper brackets? 
# As the folks at TowardsDataScience did, let's do age brackets. (And, yes, 
# this cell is blindly copying & pasting their code.)

data = [train_df, test_df]
for dataset in data:
    dataset['Age Category'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age Category'] <= 11, 'Age Category'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age Category'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age Category'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age Category'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age Category'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age Category'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age Category'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age Category'] = 6


In [ ]:
# We have to do the same for fares. 
for dataset in data:
    dataset['Fare Category'] = dataset['Fare'].astype(int)
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare Category'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare Category'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare Category']   = 2
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare Category']   = 3
    dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare Category']   = 4
    dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5

    
    
    

In [ ]:
# And, because Embarked is a single letter, let's just map it to int, too. 
# This is cheating somewhat. Instead of going through the values, and assigning
# [1, 2, 3], what we are doing is simply mapping the letter to its ordinal value.






In [ ]:
#...and now we can drop sex and age from the original data frames as we have
#the normalised columns in there. 






### Machine Learning
OK, we finally have data that we can run logistic regression on. So, here's what we have to do: 
1. Split our data frames into X_train, y_train, X_test, and y_test. This, in English, means, training data, labels for the training data, test data, and labels for test data. "X" is shorthand for training/input data, and "y" is shorthand for labels. In our case, our X will be all columns in our dataframe except Survived, and our y will be the Survived columns. 
2. Once we do that, we simply run the data as is through the sklearn LogisticRegression classifier. 

In [ ]:
# Creating our arrays, which are parameters to the LogisticRegression classifier. 







## References

- https://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8
- https://www.kaggle.com/mnassrib/titanic-logistic-regression-with-python/data#1.-Import-Data-&-Python-Packages
- https://www.kaggle.com/kernels/svzip/447794
- https://github.com/jstray/lede-algorithms/tree/master/week-3